## iii-org rancher pipeline data generator example

透過服務函式建立完整rancher pipeline資料內容(教學範例)

#### 參考
* [使用-python-讀取-yaml-檔案-d3f413d7dd6](https://medium.com/bryanyang0528/%E4%BD%BF%E7%94%A8-python-%E8%AE%80%E5%8F%96-yaml-%E6%AA%94%E6%A1%88-d3f413d7dd6)
* [yaml.readthedocs](https://yaml.readthedocs.io/en/latest/example.html)

## 引入函式庫

In [ ]:
import yaml
import sys
import os

In [6]:
## 
PIPELINE_TEMPLATE_FILE = './stander-rancher-pipeline.yaml'
## 部屬環境Config模擬
os.getenv('DEVOPS_REGISTRY', "XX.XX.XX.XX:XXXX")

'XX.XX.XX.XX:XXXX'

## 讀取標準範例樣板並顯示樣板

In [3]:
with open(PIPELINE_TEMPLATE_FILE, 'r') as stream:
    pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
yaml.dump(pipeline_template, sys.stdout)

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'true'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: 10.50.1.63:5443
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: false
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        web.type: uwsgi-flask
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: health
        db.password: bearisbear
        db.type: mysq

## 按照部屬環境進行範本環境的修改

In [5]:
def env_rancher_pipeline_generate():
    # 讀取範例檔案
    global PIPELINE_TEMPLATE_FILE
    with open(PIPELINE_TEMPLATE_FILE, 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    
    

SyntaxError: invalid syntax (<ipython-input-5-8a0234cc971a>, line 1)

In [ ]:
env_rancher_pipeline_generate()

## 使用者僅需要前端部屬

In [ ]:
def web_service_pipeline(registry_ip, registry_is_secure):
    with open('./stander-rancher-pipeline.yaml', 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最後面的階段移除
    temp_pipeline_stages.pop(2)
    # (開發環境: registry IP)
    temp_pipeline_stages[0]['steps'][0]['publishImageConfig']['registry'] = registry_ip
    return yaml.dump(pipeline_template, sys.stdout)

In [ ]:
web_service_pipeline("10.22.22.22")

## 使用者僅需要資料庫

In [ ]:
def db_service_pipeline():
    with open('./stander-rancher-pipeline.yaml', 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最前面的兩個階段移除
    temp_pipeline_stages.pop(0)
    temp_pipeline_stages.pop(0)
    return yaml.dump(pipeline_template, sys.stdout)

In [ ]:
db_service_pipeline()

In [ ]:
import pyyaml